In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import math

In [3]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

def relu(Z):
    A = np.maximum(0,Z)
    return A

In [4]:
import numpy as np
X = 2*np.random.rand(2,100)
y = 4 + np.dot([[3,11]],X)
y[y<15] = 0
y[y>=15] = 1
y = y.reshape(1,100)

In [5]:
y

array([[1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
        0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
        0., 0., 0., 1.]])

In [6]:
class Node():
    def __init__(self, input_size):
        self.input_size = input_size
        
class Layer():
   
    def __init__(self,type="hidden", activation_func="relu", n_nodes=3, input_size=None, previous_layer = None, next_layer=None):
        self.activation = activation_func
        self.n_nodes = n_nodes
        self.type = type
        self.previous_layer = previous_layer
        self.next_layer = next_layer
        """if input_size is None:
            self.input_size = previous_layer.get_output_dim()
        else:
            self.input_size = input_size"""
        self.input_size = input_size
        self.w = np.random.rand(n_nodes, input_size)
        self.a = None
        self.nodes = [Node(self.input_size) for _ in range(self.n_nodes)]
        self.loss = 0
        self.cost = 0
        self.da = 0
        self.b = np.zeros((n_nodes,1))
        self.z = 0
        self.x = 0
        self.g_prime = 0
    
    def get_output_dim(self):
        return self.n_nodes
    
    def get_output(self):
        return self.a
        
    def forward_prop(self, x, y):
        self.x = x
        #print(self.w.shape)
        #print(x.shape)
        #print("value of b is ",self.b)
        self.z = np.dot(self.w, x) + self.b ##.reshape(self.b.shape[0],1)
        
        if self.type == "output":
            self.a = relu(self.z)
            self.loss = self.a-y
            self.cost = np.sum(self.loss) / y.shape[1]
        else:
            #self.a = np.maximum(0,self.z) # for RELU
            # want to do something else in hidden layers
            self.a = sigmoid(self.z)
            self.next_layer.forward_prop(self.a, y)
            # calculate and persist Z
        #print("***Forward propogation***")
        #print("layer is ", self.type)
        #print("shape of z ",self.z.shape )
        #print("shape of a ",self.a.shape )
        #print("***Forward propogation***")
    
    def backward_prop(self, y):
        if self.type == "hidden":
            self.g_prime = self.z * (1-self.z)
            dz = self.g_prime * self.da
            dw = np.dot(dz, self.x.T) / y.shape[1]
            #db = np.squeeze(np.sum(dz, axis=1) / y.shape[1])
            db = np.sum(dz) / y.shape[1]
            #db = db.reshape((self.n_nodes,1))
            self.w = self.w - 0.01*dw
            self.b = self.b - 0.01*db
            if self.previous_layer is not None:
                self.previous_layer.da = np.dot(dw, dz)
        else:
            dz = self.a - y
            dw = np.dot(dz, self.x.T) / y.shape[1]
            db = np.squeeze(np.sum(dz, axis=1) / y.shape[1])
            self.w = self.w - 0.01*dw
            self.b = self.b - 0.01*db
            self.previous_layer.da = np.dot(dw.T, dz)
            
            self.previous_layer.backward_prop(y)
        #print("***backward propogation***")
        #print("layer is ", self.type)
        #print("shape of dz ",dz.shape )
        #print("shape of dw ",dw.shape )
        #print("shape of db ",db.shape , db)
        #if self.type == 'hidden':
            #print("shape of da ",self.da.shape )
        #print("***backward propogation***")


In [7]:
l1 = Layer(type="hidden", activation_func="relu", n_nodes=3, input_size=2, previous_layer = None, next_layer=None)
l2 = Layer(type="output", activation_func="relu", n_nodes=1, input_size=3, previous_layer = None, next_layer=None)
l1.next_layer = l2
l2.previous_layer = l1



In [8]:
l1_w_original = l1.w
l1_b_original = l1.b
l2_w_original = l2.w
l2_b_original = l2.b

In [9]:
l1_w_original,l1.w

(array([[0.08151452, 0.62773459],
        [0.41869737, 0.06059651],
        [0.07540474, 0.83782452]]),
 array([[0.08151452, 0.62773459],
        [0.41869737, 0.06059651],
        [0.07540474, 0.83782452]]))

In [10]:
for i in range(2000):
    l1.forward_prop(X,y)
    l2.backward_prop(y)
    x = i + 1
    if not x % 500:
        print("iterration ", i," ", l2.cost)

iterration  499   0.020782873774157763
iterration  999   0.02002902374926358
iterration  1499   0.019306262713977517
iterration  1999   0.018606765022804263


In [11]:
l1.w, l1_w_original

(array([[0.0767115 , 0.62833993],
        [0.40548431, 0.05994732],
        [0.10246932, 0.88754552]]),
 array([[0.08151452, 0.62773459],
        [0.41869737, 0.06059651],
        [0.07540474, 0.83782452]]))

In [12]:
l2.w, l2_w_original

(array([[0.57683974, 0.67758629, 0.67406003]]),
 array([[0.50893382, 0.98632204, 0.49848868]]))

In [13]:
l2.a[0,:10]

array([0.60673841, 0.58201423, 0.61983228, 0.40695376, 0.71821619,
       0.50660525, 0.52129264, 0.46545521, 0.7100936 , 0.48788973])

In [14]:
y[:,:10]

array([[1., 1., 1., 0., 1., 0., 1., 0., 1., 0.]])